# Clase Práctica 5: Metaheurísticas
---

## Problema

Como ejemplo estaremos resolviendo el problema del viajante (Travelling salesman problem) que requiere encontrar el camino cerrado más corto entre un conjunto de lugares.

# Colonia de Hormigas (Ant Colony Optimization)

Ant Colony Optimization (ACO) es un algoritmo de optimización utilizado para encontrar el camino más corto entre puntos o nodos. Se desarrolla observando el comportamiento de las hormigas cuando siguen un camino hacia su fuente de alimento. Las hormigas son esencialmente ciegas, por lo que siguen los rastros de feromonas que dejan otras hormigas en el camino.
Este algoritmo sigue el mismo enfoque utilizando la probabilidad de ir al siguiente nodo como la distancia al nodo y la cantidad de feromonas.

## Algoritmo

### 1. Movimiento de las hormigas según probabilidad

Para cada una de las hormigas completamos un camino cerrado, es decir, desde el inicio, cubriendo todos los nodos y sin repetir ninguno de los nodos.
Para mover una hormiga de un nodo al siguiente usamos la siguiente fórmula:
$$P_{ij}(t)=\frac{\tau_{ij}^\alpha + \eta_{ij}^\beta}{\sum (\tau^\alpha + \eta^\beta)} $$
Donde $\tau$ es la cantidad de feromonas y $\eta$ es el inverso de la distancia, es decir, $1/d$
$\alpha$ y $\beta$ son hiperparámetros del algoritmo. Se utilizan para dar más o menos peso a la distancia o a las feromonas a la hora de seleccionar el siguiente nodo. Mayor $\alpha$ hace que la hormiga dependa más de la distancia y mayor $\beta$ hace que la hormiga dependa más de las feromonas.

### 2. Depositar las feromonas

Cuando una hormiga se mueve de un nodo al siguiente, deja un rastro para que lo siga la próxima hormiga, cuantas más hormigas sigan el mismo camino, más fuerte será el rastro de feromonas. Las estelas de feromonas se incrementan en:

$$\tau^{(i+1)} = \tau^i + \Delta\tau $$ 

Donde $\Delta\tau$ (*delta_tau*) es un parámetro e $i$ es el número de iteraciones.

### 3. Evaporación de las feromonas

Después de cada iteración, las feromonas también tienden a evaporarse. La evaporación de las feromonas se da como:
$$\tau^{(i+1)} = (1 - \rho)\tau^i $$ 
donde $\rho$ es la tasa de evaporación.

### 4. Condición de parada

Podemos usar cualquier condición para finalizar la búsqueda, como una distancia por debajo de un cierto umbral o el número de iteraciones.

In [5]:
import random as rn
import numpy as np
from numpy.random import choice as np_choice

class AntColony(object):

    def __init__(self, distances, n_ants, n_iterations, decay, alpha=1, beta=0.5, delta_tau = 2):
        self.distances  = distances
        self.pheromone = np.zeros(self.distances.shape)
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta
        self.delta_tau = delta_tau
        
    def run(self):
        shortest_path = None
        all_time_shortest_path = (-1, "placeholder", np.inf)
        
        for i in range(self.n_iterations):
            all_paths = self.gen_all_paths()
            self.spread_pheronome(all_paths)
            shortest_path = min(all_paths, key=lambda x: x[1])
            
            if shortest_path[1] < all_time_shortest_path[2]:
                all_time_shortest_path = (i, *shortest_path)    
            
            if i%10==0: print(i,  "mean: ", mean([path[1] for path in all_paths]), "best_iteration_solution: ", shortest_path ,"best_global_solution: ", all_time_shortest_path)
            
        return all_time_shortest_path

    def spread_pheronome(self, all_paths):
        self.evaporate_pheromone()
        for path, dist in all_paths:
            for i in range(len(path)-1):
                self.pheromone[path[i], path[i+1]] += self.delta_tau/dist
        return None

    def path_dist(self, path):
        dist = 0
        for i in range(len(path)-1):
            dist += self.distances[path[i], path[i+1]]
        return dist


    def gen_all_paths(self):
        all_paths = []
        for i in range(self.n_ants):
            path = self.gen_path(0)
            all_paths.append((path, self.path_dist(path)))
        return all_paths

    def gen_path(self, start):
        path = [start]
        visited = [start]
        while len(visited) < self.distances.shape[0]:
            move = self.pick_move(self.pheromone, self.distances, visited)
            path.append(move)
            visited.append(move)
        return path

    def pick_move(self, pheromone, dist, visited):
        pheromone = pheromone[visited[-1], :]
        pheromone[visited] = 0
        pheromone = pheromone**self.alpha
        dist = dist[visited[-1], :]
        dist[visited] = np.inf
        dist = dist**self.beta
        probs = pheromone/dist
        probs = probs/sum(probs)
        move = np_choice(range(len(probs)), p=probs)
        return move
    
    def evaporate_pheromone(self):
        self.pheromone = self.pheromone * self.decay
        return None


In [6]:
distances = np.array([[np.inf, 2, 2, 5, 7],

                      [2, np.inf, 4, 8, 2],

                      [2, 4, np.inf, 1, 3],

                      [5, 8, 1, np.inf, 2],

                      [7, 2, 3, 2, np.inf]])

In [7]:
ant_colony = AntColony(distances, 5, 100, 0.95, alpha=1, beta=1, delta_tau = 2)

In [8]:
ant_colony.run()

/var/folders/cm/y1ylnx7j2xzd3ty4g05zq63w0000gn/T/ipykernel_7194/893830802.py:71: RuntimeWarning: invalid value encountered in divide
  probs = probs/sum(probs)


ValueError: probabilities contain NaN

# Recocido Simulado (Simulated Annealing)

La noción de enfriamiento lento implementada en el algoritmo de recocido simulado se interpreta como una disminución lenta en la probabilidad de aceptar peores soluciones a medida que se explora el espacio de soluciones. Aceptar peores soluciones permite una búsqueda más extensa de la solución óptima global. En general, los algoritmos de recocido simulado funcionan de la siguiente manera. La temperatura desciende progresivamente desde un valor inicial positivo hasta cero. En cada paso de tiempo, el algoritmo selecciona aleatoriamente una solución cercana a la actual, mide su calidad y se mueve hacia ella de acuerdo con las probabilidades dependientes de la temperatura de seleccionar mejores o peores soluciones, que durante la búsqueda permanecen respectivamente en 1 (o positivo). ) y disminuye hacia cero.

Para la implementacion nos apoyamos de las siguientes funciones:

## Sucesor

Toma un estado y devuelve un nuevo estado aleatorio después de hacer alguna transformación al estado anterior. Una propiedad importante de la función sucesor es que no cambia demasiado la respuesta. Otra propiedad importante es que debería ser posible llevar cualquier estado a cualquier otro estado en un número corto de pasos.

## Temperatura

La función de temperatura decide qué tan "dispuesto" estará el algoritmo para pasar a un estado peor. En otras palabras, si la temperatura es 0, solo se movería a mejores estados (lo que se convierte en el algoritmo de hill climbing). Si la temperatura es infinita, te moverías a cualquier estado, sin importar qué tan bueno o malo sea. En el recocido simulado, desea que la temperatura pase de algo alto a algo bajo. Inicialmente, desea poder explorar los diferentes estados posibles, de modo que tenga una mejor oportunidad de alcanzar el mínimo global. Al final, desea seguir tomando mejores y mejores estados, esperando que ya esté cerca del mínimo global. Hay 3 reglas comunes de reducción de temperatura:

- $ t=t⋅a $
- $ t=t−a $
- $ t=t /(1+a⋅t) $

donde $a$ es un parámetro.

Una de las más comunes es la primera, empezando con valores de $t_0 = 10^9$
o $t_0 = 10^5$ y $a=0.99999$.

## Probabilidad de aceptación

La más usada para recocido simulado es [Metropolis-Hastings Algorithm](!https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm) que se calcula de la siguiente forma:
```
def P(old, new, temp):
    if new < old:
        return 1.0
    return exp((old-new)/temp)
```
donde `old` y `new` son los valores de fitness del actual estado y el sucesor respectivamente

In [ ]:
import random
import math

class SimulatedAnnealing(object):

    def __init__(self, distances, n_iterations, a = 0.99, t_0= 10**9):
        self.distances  = distances
        self.n_iterations = n_iterations
        self.a = a
        self.t_0 = t_0
        
    def run(self):
        current_path = self.gen_random_path()
        all_time_shortest_path = (-1, "placeholder", np.inf)
        t = self.t_0
        
        for i in range(self.n_iterations):
            next_path = self.succesor(current_path)
            print(next_path)
            if self.path_dist(current_path) < all_time_shortest_path[2]:
                all_time_shortest_path = (i, current_path ,self.path_dist(current_path))  
            
            if self.acceptance_probability(self.path_dist(current_path), self.path_dist(next_path), t) >= random.random():
                path = next
                
            t = self.temperature(t)
            
            print(i, "best_solution: ", all_time_shortest_path)
            
        return all_time_shortest_path
    
    def path_dist(self, path):
        #TODO: Your code here!
        return None

    def gen_random_path(self):
        #TODO: Your code here!
        return None
                         
    def temperature(self, t):
        #TODO: Your code here!
        return None
                         
    def acceptance_probability(self, current_dist, next_dist, temp):
        #TODO: Your code here!
        return None
    
    def succesor(self, path):
        #TODO: Your code here!
        return None
        

In [ ]:
sa = SimulatedAnnealing(distances, 100)

In [ ]:
sa.run()